In [1]:
#Read Data
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vYdLFvS7OIFtrvWEDBnshv8-8wT-1U2HCqOLpQTFwYFK',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.jp-tok.cloud-object-storage.appdomain.cloud')

#daily report
bucket = 'forex-donotdelete-pr-z4qvpns5xgagyh'
object_key = 'daily_report.csv'
body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_1 = pd.read_csv(body)

#trades
object_key = 'trades.csv'
body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_2 = pd.read_csv(body)

#login
object_key = 'login.csv'
body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_3 = pd.read_csv(body)

In [2]:
#Clean Data
df_2['open_time'] = pd.to_datetime(df_2['open_time'], unit='s')
df_2['open_time'] = df_2['open_time'].dt.tz_localize('UTC')
df_2['open_time'] = df_2['open_time'].dt.tz_localize(None)
df_2['close_time'] = pd.to_datetime(df_2['close_time'], unit='s')
df_2['close_time'] = df_2['close_time'].dt.tz_localize('UTC')
df_2['close_time'] = df_2['close_time'].dt.tz_localize(None)
df_2 = pd.merge(df_2, df_3[['login', 'account_currency']], on='login', how='left')

In [3]:
#Create Junction Table for changing profit to USD
currency_abreviation = df_2['account_currency'].tolist()
curr_to_US = ["AUD", "EUR", "GBP", "NZD"]
curr_from_US = ["CAD", "CHF", "CNH", "HKD", "HUF", "JPY", "MXN", "NLK", "PLN", "SEK", "SGD", "THB", "TRY", "ZAR"]
curr_US = ["USD"]
curr_reader = []

for i in range(len(currency_abreviation)):
    for j in range(len(curr_to_US)):
        if currency_abreviation[i] == curr_to_US[j]:
            curr_csv = str(curr_to_US[j] + "USD.csv")
            curr_reader.append(curr_csv)
    for x in range(len(curr_from_US)):
        if currency_abreviation[i] == curr_from_US[x]:
            curr_csv = str("USD.csv" + curr_from_US[x])
            curr_reader.append(curr_csv)
    for y in range(len(curr_US)):
        if currency_abreviation[i] == curr_US[y]:
            curr_csv = str("USDUSD")
            curr_reader.append(curr_csv)

close_dates = df_2['close_time'].apply(lambda x: x.strftime('%Y.%m.%d')).tolist()
ticket_exch = df_2['ticket'].tolist()
profit_changer = list(zip(ticket_exch, curr_reader, close_dates))